In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [51]:
df = pd.read_csv('energydata_complete.csv')
df.columns

Index(['date', 'Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
       'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')

In [52]:
scaler = MinMaxScaler()

In [53]:
df.drop(columns = ['date','lights'], inplace=True)

In [54]:
col_names = df.columns

In [55]:
normalised_df = pd.DataFrame(scaler.fit_transform(df), columns = col_names)

C:\Users\Bolaji\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [56]:
predictor = normalised_df[['T2']]
predictor2 = normalised_df.drop(columns = ['Appliances'])

In [57]:
target = normalised_df[['T6']]
target_appliance = normalised_df[['Appliances']]

In [58]:
x_train, x_test, y_train, y_test = train_test_split(predictor, target, test_size = 0.3, random_state = 42)

In [59]:
linear_model = LinearRegression()

In [60]:
linear_model.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [61]:
linear_predictions = linear_model.predict(x_test)

In [62]:
linear_residuals = y_test - linear_predictions

In [63]:
linear_r2_score = round(r2_score(y_test, linear_predictions), 2)
print(round(linear_r2_score, 2))

0.64


In [64]:
linear_mae = mean_absolute_error(y_test, linear_predictions)
print(round(linear_mae, 2))

0.08


In [65]:
linear_rss = np.sum(np.square(linear_residuals))
print(round(linear_rss, 2))

T6    66.12
dtype: float64


In [66]:
linear_rmse = np.sqrt(mean_squared_error(y_test, linear_predictions))
print(round(linear_rmse, 3))

0.106


In [67]:
x2_train,x2_test,y2_train,y2_test = train_test_split(predictor2, target_appliance, test_size = 0.3, random_state = 42)

In [68]:
ridge_model = Ridge(alpha = 0.4)

In [69]:
ridge_model.fit(x2_train,y2_train)

Ridge(alpha=0.4, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [76]:
ridge_predictions = ridge_model.predict(x2_test)

In [78]:
ridge_rmse = np.sqrt(mean_squared_error(y2_test, ridge_predictions))
print(ridge_rmse)

0.08753385704628003


In [72]:
lasso_model = Lasso(alpha = 0.001)

In [73]:
lasso_model.fit(x2_train,y2_train)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [74]:
lasso_predictions = lasso_model.predict(x2_test)

In [79]:
lasso_rmse = np.sqrt(mean_squared_error(y2_test, lasso_predictions))
print(round(lasso_rmse, 3))

0.094


In [80]:
def get_weights_df(model, feat, col_name) :
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = [ 'Features' , col_name]
    weights_df[col_name].round( 3 )
    return weights_df

In [81]:
lasso_weights = get_weights_df(lasso_model,predictor2, 'Lasso Weight')

In [83]:
lasso_weights

,Features,Lasso Weight
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000
